# AWS Lambda with Boto3

### Setting Up Project

In [ ]:
import boto3
import json

In [ ]:
from os import path
from utils import Utils

In [ ]:
LAMBDA_ACCESS_POLICY_ARN = 'arn:aws:iam::059062539581:policy/LambdaS3AccessPolicy'
LAMBDA_ROLE = 'Lambda_Execution_Role'
LAMBDA_ROLE_ARN = 'arn:aws:iam::059062539581:role/Lambda_Execution_Role'
LAMBDA_TIMEOUT = 10
LAMBDA_MEMORY = 128
LAMBDA_HANDLER = 'lambda_function.handler'
PYTHON_36_RUNTIME = 'python3.6'
NODEJS_810_RUNTIME = 'nodejs10.x'
JAVA_8_RUNTIME = 'java8'
NODEJS_LAMBDA_NAME = 'NodeJSLambdaFunction'
PYTHON_LAMBDA_NAME = 'PythonLambdaFunction'
JAVA_LAMBDA_NAME = 'JavaLambdaFunction'

### Create boto3 aws client

In [ ]:
def lambda_client():
    aws_lambda = boto3.client('lambda', region_name='eu-west-1')
    """ :type : pyboto3.lambda """
    return aws_lambda

### Create aws iam cient to create role and access policy

In [ ]:
def iam_client():
    iam = boto3.client('iam')
    """ :type : pyboto3.iam """
    return iam

### Creating an IAM Lambda Access Policy

In [ ]:
def create_access_policy_for_lambda():
    s3_access_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Action": [
                    "s3:*",
                    "logs:CreateLogGroup",
                    "logs:CreateLogStream",
                    "logs:PutLogEvents"
                ],
                "Effect": "Allow",
                "Resource": "*"
            }
        ]
    }

    return iam_client().create_policy(
        PolicyName='LambdaS3AccessPolicy',
        PolicyDocument=json.dumps(s3_access_policy_document),
        Description='Allows lambda function to access S3 resources'
    )

### Creating an IAM Execution Role

In [ ]:
def create_execution_role_for_lambda():
    lamda_execution_assumption_role = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    return iam_client().create_role(
        RoleName=LAMBDA_ROLE,
        AssumeRolePolicyDocument=json.dumps(lamda_execution_assumption_role),
        Description="Gives necessary permissions for lambda to be executed"
    )

### Attaching IAM Access Policy to IAM Execution Role

In [ ]:
def attach_access_policy_to_execution_role():
    return iam_client().attach_role_policy(
        RoleName=LAMBDA_ROLE,
        PolicyArn=LAMBDA_ACCESS_POLICY_ARN
    )

### Implementing Utility Methods for Source Code Bytes Reading

In [ ]:
cat utils.py

### Developing a Python Lambda Function

In [ ]:
cat python_lambda_function.py

### Deploying Python Lambda Function

In [ ]:
def deploy_lambda_function(function_name, runtime, handler, role_arn, source_folder):
    print (f"source_folder: {source_folder}")
    print (f"function_name: {function_name}")
    print (f"role_arn: {role_arn}")
    print (f"runtime: {runtime}")
    
    folder_path = path.join(path.abspath("."), source_folder)

    print (f"folder_path: {folder_path}")
    
    if runtime is not JAVA_8_RUNTIME:
        zip_file = Utils.make_zip_file_bytes(path=folder_path)
    else:
        zip_file = Utils.read_jar_file(folder_path)

    print (f"zip_file: {zip_file}")
    
    return lambda_client().create_function(
        FunctionName=function_name,
        Runtime=runtime,
        Role=role_arn,
        Handler=handler,
        Code={
            'ZipFile': zip_file
        },
        Timeout=LAMBDA_TIMEOUT,
        MemorySize=LAMBDA_MEMORY,
        Publish=False
    )


### AWS Console Checkpoint: Python Lambda Function
 - Open AWS Console and verify that lambda function is created as expected

# Updating Function Code and Versioning

### Invoking Functions

In [ ]:
def invoke_lambda_function(function_name):
    return lambda_client().invoke(FunctionName=function_name)

### Modify lambda function configuration to accept environment variables

In [ ]:
def add_environment_variables_to_lambda(function_name, variables):
    return lambda_client().update_function_configuration(
        FunctionName=function_name,
        Environment=variables
    )


### Modifying Function Code to Use Environment Variable and Invoking

In [ ]:
def update_lambda_function_code(function_name, source_folder):
    folder_path = path.join(path.abspath('.'), source_folder)
    zip_file = Utils.make_zip_file_bytes(path=folder_path)

    return lambda_client().update_function_code(
        FunctionName=function_name,
        ZipFile=zip_file
    )

### Publish new version of lambda

In [ ]:
def publish_a_new_version(function_name):
    return lambda_client().publish_version(
        FunctionName=function_name
    )


### Creating Aliases for Function Versions

In [ ]:
def create_alias_for_new_version(function_name, alias_name, version):
    return lambda_client().create_alias(
        FunctionName=function_name,
        Name=alias_name,
        FunctionVersion=version,
        Description='This is the ' + alias_name + ' alias for function'
    )

In [ ]:
def invoke_lambda_with_alias(function_name, alias_name):
    return lambda_client().invoke(
        FunctionName=function_name,
        Qualifier=alias_name
    )

# Function Operations

### Getting a Function Configuration

In [ ]:
def get_function(function_name):
    return lambda_client().get_function(FunctionName=function_name)

### Listing All Function Configurations

In [ ]:
def get_all_functions():
    return lambda_client().list_functions()

### Updating Function Configurations

In [ ]:
def increase_lambda_execution_memory(function_name, new_memory_size):
    return lambda_client().update_function_configuration(
        FunctionName=function_name,
        MemorySize=new_memory_size
    )

### Deleting Functions

In [ ]:
def delete_lambda_function(function_name):
    return lambda_client().delete_function(FunctionName=function_name)

In [ ]:
# print(create_access_policy_for_lambda())


In [ ]:
# print(create_execution_role_for_lambda())


In [ ]:
# print(attach_access_policy_to_execution_role())


In [ ]:
# print(deploy_lambda_function(PYTHON_LAMBDA_NAME, PYTHON_36_RUNTIME, LAMBDA_HANDLER, LAMBDA_ROLE_ARN, 'python_lambda'))


In [ ]:
# print(deploy_lambda_function(NODEJS_LAMBDA_NAME, NODEJS_810_RUNTIME, LAMBDA_HANDLER, LAMBDA_ROLE_ARN, 'nodejs_lambda'))


In [ ]:
# print(deploy_lambda_function(JAVA_LAMBDA_NAME, JAVA_8_RUNTIME, 'com.amazonaws.lambda.demo.LambdaFunctionHandler::handleRequest', LAMBDA_ROLE_ARN, 'JavaLambdaFunction/target/demo-1.0.0.jar'))


In [ ]:
# response = invoke_lambda_function(PYTHON_LAMBDA_NAME)


In [ ]:
#print(response['Payload'].read().decode())


In [ ]:
env_variables = {
    'Variables': {
        'ENV_VAR_TEST': 'This is an environment variable!'
    }
}


In [ ]:
#add_environment_variables_to_lambda(PYTHON_LAMBDA_NAME, env_variables)


In [ ]:
#print(update_lambda_function_code(PYTHON_LAMBDA_NAME, 'python_lambda'))


In [ ]:
response = invoke_lambda_function(PYTHON_LAMBDA_NAME)
print(response['Payload'].read().decode())


In [ ]:
print(publish_a_new_version(PYTHON_LAMBDA_NAME))


In [ ]:
create_alias_for_new_version(PYTHON_LAMBDA_NAME, 'PROD', '1')


In [ ]:
response = invoke_lambda_with_alias(PYTHON_LAMBDA_NAME, 'PROD')
print(response['Payload'].read().decode())


In [ ]:
print(get_function(PYTHON_LAMBDA_NAME))


In [ ]:
print(get_all_functions())


In [ ]:
increase_lambda_execution_memory(PYTHON_LAMBDA_NAME, 256)


In [ ]:
delete_lambda_function(NODEJS_LAMBDA_NAME)